# Exemple Iterator Pattern : Restaurant Menus

Exemple repris depuis HeadFirst Design Pattern (en JAVA) : deux restaurant se réunissent pour n'en faire qu'un. L'un a implémenté un menu en ArrayMist et le second en Array.  
La serveuse doit être capable de donner au client tous les items de la carte, seulement ceux du petit dès ou du déjeuner, ou seulement les options végé. Comment faire puisque les implémentations des restaurants sont différentes ? 

__Le but est que la serveuse puiss itérer dans les menus sans être dépendante de la façon dont les menus sont implémentés.__

# Menus et Serveuse 1.0

Les MenuItem sont identiques dans les deux restaurants.

## MenuItem

In [9]:
class MenuItem:

    name: str
    description: str
    vegetarian: bool
    price: float

    def __init__(self, name: str, description: str, vegetarian: bool, price: float):
        self.name = name
        self.description = description
        self.vegetarian = vegetarian
        self.price = float

## Définition des menus 

In [ ]:
class PancakeHouseMenu:
    
    def __init__(self) -> None:
        self.menuItems=[]
        self.addItem("K&B's Pancake Breakfast", "Pancakes with scrambled eggs, and toast", True, 2.99)
        self.addItem("Regular Pancake Breakfast", "Pancakes with fried eggs, sausage", False, 2.99)
        self.addItem("Blueberry Pancake", "Pancakes made with fresh blueberries", True, 3.49)
        self.addItem("Waffles", "Waffles, with your choice of blueberries or strawberries", True, 3.59)
        
    def addItem(self, name:str, description:str, vegetarian:bool, price:float) -> None:
        self.menuItem = MenuItem(name, description, vegetarian, price)
        self.menuItems.append(self.menuItem)
        
    def getMenuItems(self):
        return self.menuItems

In [ ]:
class DinerMenu:
    MAX_ITEMS: int = 6
    numberOfItems: int = 0
    
    def __init__(self):
        self.menuItems=[None]*self.MAX_ITEMS
        self.addItem("Vegetarian BLT","(Fakin') Bacon with Lettuce & tomato on whole wheat", True, 2.99)
        self.addItem("BLT","Bacon with Lettuce & tomato on whole wheat", False, 2.99)
        self.addItem("Soup of the day","Soup of the day with a side of potato salad", False, 3.29)
        self.addItem("Hotdog","Soup of the day with a side of potato salad", False, 3.29)
        
    def addItem(self, name:str, description:str, vegetarian:bool, price:float) -> None:
        self.menuItem = MenuItem(name, description, vegetarian, price)
        if self.numberOfItems >= self.MAX_ITEMS:
            print("Sorry, menu is full! Can't add item to menu.")
        else:
            self.menuItems[self.numberOfItems] = self.menuItem;
            self.numberOfItems += 1
        
    def getMenuItems(self):
        return self.menuItems
    

Tests des méthodes

In [ ]:
pHM = PancakeHouseMenu()
petitDej = pHM.getMenuItems()
for item in petitDej:
    print(item.name)

In [ ]:
dM = DinerMenu()
lunch = dM.getMenuItems()
for item in lunch:
    try:
        print(item.name)
    except:
        pass

## Serveuse

La serveuse (Alice) doit savoir imprimer les menus suivants : 
* printMenu() : tous les items
* printBreakfastMenu : seulement le petit dèj
* printLunchMenu : prints just lunch items
* printVegetarianMenu : prints all vegetarian menu items
* isItemVegetarian(name) : sait dire si l'item passé en paramètre est végé ou non

In [ ]:
class Waitress:
    pHM = PancakeHouseMenu()
    breakfastItems = pHM.getMenuItems()
    dM = DinerMenu()
    lunchItems = dM.getMenuItems()
    
    def printMenu(self):
        for menuItem in self.breakfastItems:
            try:
                print(menuItem.name)
            except:
                pass
        for menuItem in self.lunchItems:
            try:
                print(menuItem.name)
            except:
                pass
            
    def printBreakfastMenu(self):
        for menuItem in self.breakfastItems:
            try:
                print(menuItem.name)
            except:
                pass
            
    def printVegetarianMenu(self):
        for menuItem in self.breakfastItems:
            try:
                if menuItem.vegetarian :print(menuItem.name)
            except:
                pass
            
        for menuItem in self.lunchItems:
            try:
                if menuItem.vegetarian: print(menuItem.name)
            except:
                pass

In [ ]:
Alice = Waitress()
Alice.printMenu()

In [ ]:
Alice.printBreakfastMenu()

In [ ]:
Alice.printVegetarianMenu()

# Menus et serveuse 2.0

Maintenant on veut une nouvelle Waitress qui ne repose pas autant sur les implémentations des menus de chacun des restaurants.  Pour cela on va créer des itérateurs pour chacun des menus, et on va modifier la serveuse pour qu'elle soit capable d'utiliser ces itérateurs.

## Iterator
On utilise l'interface iterator de Python, que les menus vont implémenter. Cette classe dispose de :

* 2 attributs :  
    * ___position__ : entier représentant la position actuelle de l'item (HAP : utile dans notre cas ?!)
    * ___reverse__ : booleen indiquant le sens de parcours de la collection
    
    
* 2 méthodes :
    * __\_\_init\_\___ : set la collection dans laquelle itérer, le sens et l'incrément d'itération.
    * __\_\_next\_\___ : méthode renvoyant l'item suivant d'une séquence ou lève une erreur si fin de la séquence

In [5]:
from __future__ import annotations
from collections.abc import Iterable, Iterator

In [6]:
class DinerMenuIterator(Iterator):
    _position: int = None
    _reverse: bool = False
        
    def __init__(self, collection:[], reverse: bool = False) -> None:
        self._collection = collection
        self._reverse = reverse
        self._position = -1 if reverse else 0

    def __next__(self):
        try:
            value = self._collection[self._position]
            self._position += -1 if self._reverse else 1
        except IndexError:
            raise StopIteration()
            
        return value

In [35]:
class PancackeHouseMenuIterator(Iterator):
    _position: int = None
    _reverse: bool = False
        
    def __init__(self, collection:[], reverse: bool = False) -> None:
        self._collection = collection
        self._reverse = reverse
        self._position = -1 if reverse else 0

    def __next__(self):
        for value in self._collection:
            elf._position += -1 if self._reverse else 1
            return value

In [7]:
class DinerMenu():
    MAX_ITEMS: int = 6
    numberOfItems: int = 0
    
    def __init__(self):
        self.menuItems=[None]*self.MAX_ITEMS
        self.addItem("Vegetarian BLT","(Fakin') Bacon with Lettuce & tomato on whole wheat", True, 2.99)
        self.addItem("BLT","Bacon with Lettuce & tomato on whole wheat", False, 2.99)
        self.addItem("Soup of the day","Soup of the day with a side of potato salad", False, 3.29)
        self.addItem("Hotdog","Soup of the day with a side of potato salad", False, 3.29)
        
    def addItem(self, name:str, description:str, vegetarian:bool, price:float) -> None:
        self.menuItem = MenuItem(name, description, vegetarian, price)
        if self.numberOfItems >= self.MAX_ITEMS:
            print("Sorry, menu is full! Can't add item to menu.")
        else:
            self.menuItems[self.numberOfItems] = self.menuItem;
            self.numberOfItems += 1
        
    def createIterator(self):
        return DinerMenuIterator(self.menuItems)

In [ ]:
class PancakeHouseMenu:
    
    def __init__(self) -> None:
        self.menuItems=[]
        self.addItem("K&B's Pancake Breakfast", "Pancakes with scrambled eggs, and toast", True, 2.99)
        self.addItem("Regular Pancake Breakfast", "Pancakes with fried eggs, sausage", False, 2.99)
        self.addItem("Blueberry Pancake", "Pancakes made with fresh blueberries", True, 3.49)
        self.addItem("Waffles", "Waffles, with your choice of blueberries or strawberries", True, 3.59)
        
    def addItem(self, name:str, description:str, vegetarian:bool, price:float) -> None:
        self.menuItem = MenuItem(name, description, vegetarian, price)
        self.menuItems.append(self.menuItem)
        
    def getMenuItems(self):
        return self.menuItems

In [31]:
class Waitress2:
    #pHM = PancakeHouseMenu()
    dM = DinerMenu()
    bM
    
    def printMenu(self):
        dinerIterator = self.dM.createIterator()
        print("Impression du menu du diner via iterateur :")
        self.printingMenu(dinerIterator)
        
    def printBreakfastMenu(self):
        
    def printVegetarianMenu(self):
    
    def printingMenu(self, iterator:DinerMenuIterator):
        for menuItem in iterator:
            try:
                print(menuItem.name)
            except:
                pass

In [32]:
Mary = Waitress2()

In [33]:
Mary.printMenu()

Impression du menu du diner via iterateur :
Vegetarian BLT
BLT
Soup of the day
Hotdog


In [ ]:
class AlphabeticalOrderIterator(Iterator):

    _position: int = None
    _reverse: bool = False

    def __init__(self, collection: WordsCollection, reverse: bool = False) -> None:
        self._collection = collection
        self._reverse = reverse
        self._position = -1 if reverse else 0

    def __next__(self):
        try:
            value = self._collection[self._position]
            self._position += -1 if self._reverse else 1
        except IndexError:
            raise StopIteration()

        return value

In [ ]:
class DinerMenu(Iterator):
    MAX_ITEMS: int = 6
    numberOfItems: int = 0
    
    def __init__(self):
        self.menuItems=[None]*self.MAX_ITEMS
        self.addItem("Vegetarian BLT","(Fakin') Bacon with Lettuce & tomato on whole wheat", True, 2.99)
        self.addItem("BLT","Bacon with Lettuce & tomato on whole wheat", False, 2.99)
        self.addItem("Soup of the day","Soup of the day with a side of potato salad", False, 3.29)
        self.addItem("Hotdog","Soup of the day with a side of potato salad", False, 3.29)
        
    def addItem(self, name:str, description:str, vegetarian:bool, price:float) -> None:
        self.menuItem = MenuItem(name, description, vegetarian, price)
        if self.numberOfItems >= self.MAX_ITEMS:
            print("Sorry, menu is full! Can't add item to menu.")
        else:
            self.menuItems[self.numberOfItems] = self.menuItem;
            self.numberOfItems += 1
        
    def getMenuItems(self):
        return self.menuItems
    

## Structure du Pattern

![Image structure du Pattern Iterator](Images/IteratorPattern.PNG)

## Diagramme de classe de l'exemple

Ici dans notre exemple les classes abstraites Iterator et Iterable que l'on implémente sont déjà définies en Python et les classes codées ici sont les classes concrètes.  
On passe en paramètre à l'itérateur une collection de mots, que l'on va ensuite parcourir dans un sens ou dans l'autre.

![Diagramme de classe de Iterator Pattern](Images/IteratorPatternClass.PNG)

## Code

In [ ]:
from __future__ import annotations
from collections.abc import Iterable, Iterator
from typing import Any, List

### Classe Iterator en ordre alphabétique

Classe iterator dispose de 2 attributs : 
* __\_position__ : entier représentant la position actuelle de l'item
* __\_reverse__ : booleen indiquant le sens de parcours de la collection

Classe iterator implémente les méthodes :
* __\_\_init\_\___ : set la collection dans laquelle itérer, le sens et l'incrément d'itération.
* __\_\_next\_\___ : méthode renvoyant l'item suivant d'une séquence ou lève une erreur si fin de la séquence

### Classe WordsCollection

La collection implémente l'interface Iterable. Elle dispose des méthodes suivantes :  
* __\_\_init\_\___ : attribue à \_collection la liste passée en paramètres 
* __\_\_iter\_\___ : méthode qui renvoie la collection dans laquelle iterer. Il s'agit d'une instance de AlphabeticalOrderIterator : crée cette instance, en passant en paramètre sa collection de mots.
* __get_reverse_iterator__ : similaire à __\_\_iter_\_\___ sauf que création d'une instance de AlphabeticalOrderIterator en lui passant en paramètre la collection de mots en ordre inverse.
* __add_item__ : permet d'ajouter un élément à la collection.

In [ ]:
class WordsCollection(Iterable):

    def __init__(self, collection: List[Any] = []) -> None:
        self._collection = collection

    def __iter__(self) -> AlphabeticalOrderIterator:
        return AlphabeticalOrderIterator(self._collection)

    def get_reverse_iterator(self) -> AlphabeticalOrderIterator:
        return AlphabeticalOrderIterator(self._collection, True)

    def add_item(self, item: Any):
        self._collection.append(item)

## Code Test

Pour tester notre itérator, on crée une instance de WordCollection() à laquelle on ajoute des mots.  
Puisqu'elle implémente iterator, et donc __\_\_iter\_\___, WordCollection est "itérable".

La méthode _join()_ prend tous les items d'un iterable et les joint dans une chaîne unique avec la syntaxe suivante : 
separateur.join(iterable).

Par exemple : 
```python
liste = ["Un", "Deux", "Trois"]
separateur = " espace "
separateur.join(liste)
```

A pour résultat : 
```python
'Un espace Deux espace Trois'
```

Dans l'exemple repris depuis RefactoringGuru, le séparateur est un saut de ligne.

In [ ]:
collection = WordsCollection()
collection.add_item("First")
collection.add_item("Second")
collection.add_item("Third")

print("Straight traversal:")
print("\n".join(collection))
print("")

print("Reverse traversal:")
print("\n".join(collection.get_reverse_iterator()), end="")